In [ ]:
##graph2graph test script
#imports
import multiprocessing
from multiprocessing import Pool
import math, random, sys
import pickle
import argparse
from functools import partial
import torch
import numpy

from hgraph import MolGraph, common_atom_vocab, PairVocab
import rdkit
import rdkit.Chem as Chem
import networkx as nx
from hgraph.chemutils import *
from hgraph.nnutils import *

In [16]:
#check to make sure GPU is available
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-16GB'

In [17]:
#get vocab from get_vocab.py (previous step)
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

with open('data/chembl/cyclic_peptide_vocab_truncated.txt') as f:
    vocab = [x.strip("\r\n ").split() for x in f]

vocab = PairVocab(vocab, cuda=False)

pool = Pool(multiprocessing.cpu_count()) 
random.seed(1)

Process ForkPoolWorker-28:
Process ForkPoolWorker-21:
Process ForkPoolWorker-3:
Process ForkPoolWorker-24:
Process ForkPoolWorker-40:
Process ForkPoolWorker-26:
Process ForkPoolWorker-4:
Process ForkPoolWorker-39:
Process ForkPoolWorker-22:
Process ForkPoolWorker-17:
Process ForkPoolWorker-20:
Process ForkPoolWorker-16:
Process ForkPoolWorker-12:
Process ForkPoolWorker-18:
Process ForkPoolWorker-23:
Process ForkPoolWorker-8:
Process ForkPoolWorker-37:
Process ForkPoolWorker-27:
Process ForkPoolWorker-6:
Process ForkPoolWorker-10:
Process ForkPoolWorker-35:
Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Process ForkPoolWorker-11:
Process ForkPoolWorker-7:
Process ForkPoolWorker-36:
Process ForkPoolWorker-15:
Process ForkPoolWorker-1:
Process ForkPoolWorker-38:
Process ForkPoolWorker-14:
Process ForkPoolWorker-25:
Process ForkPoolWorker-9:
Process ForkPoolWorker-13:
Process ForkPoolWorker-19:
Process ForkPoolWorker-33:
Process ForkPoolWorker-34:
Process ForkPoolWorker-29:
Process Fo

  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args

  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "

  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/multiprocessing/synch

In [18]:
vocab

In [19]:
with open('data/chembl/cyclic_peptide_smiles_truncated.txt') as f:
    data = [line.strip("\r\n ").split()[0] for line in f]

random.shuffle(data)
batch_size = 32

batches = [data[i : i + batch_size] for i in range(0, len(data), batch_size)]
# func = partial(tensorize, vocab = vocab)
# all_data = pool.map(func, batches)
# num_splits = len(all_data) // 1000

# le = (len(all_data) + num_splits - 1) // num_splits

# for split_id in range(num_splits):
#     st = split_id * le
#     sub_data = all_data[st : st + le]

#     with open('tensors-%d.pkl' % split_id, 'wb') as f:
#         pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)

In [22]:
batches[0]

['N[C@@H](CC(=O)N)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](C)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CS3)C(=O)N[C@@H](CO)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](C)C(=O)N[C@@H](C(C)C)C(=O)N1[C@@H](CCC1)C(=O)N[C@@H](CS3)C(=O)N[C@@H]([C@]([H])(CC)C)C(=O)O',
 'N[C@@H](CC(=CN2)C1=C2C=CC=C1)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CS3)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](Cc1ccccc1)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CS3)C(=O)N1[C@@H](CCC1)C(=O)O',
 'N[C@@H](CC(C)C)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(=CN2)C1=C2C=CC=C1)C(=O)N[C@@H](CSCC(C4)=O)C(=O)N[C@@H](CS3)C(=O)N[C@@H](Cc1ccccc1)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CS4)C(=O)N1[C@@H](CCC1)C(=O)O',
 'N[C@@H](C)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](CSCC(C4)=O)C(=O)N[C@@H](CCCCN)C(=O)N[C@@H](CO)C(=O)N[C@@H]([C@]([H])(CC)C)C(=O)N[C@@H](CS3)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CS4)C(=O)N1[C@@H](CCC1)C(=O)O',
 'N[C@@H](CO)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](CO)C(=O)N[C@@H](CSC

In [26]:
#tensorize function, as given in the hgraph2graph package
def tensorize(mol_batch, vocab, avocab):
    mol_batch = [MolGraph(x) for x in mol_batch]
    tree_tensors, tree_batchG = MolGraph.tensorize_graph([x.mol_tree for x in mol_batch], vocab)
    graph_tensors, graph_batchG = MolGraph.tensorize_graph([x.mol_graph for x in mol_batch], avocab)
    tree_scope = tree_tensors[-1]
    graph_scope = graph_tensors[-1]

    max_cls_size = max( [len(c) for x in mol_batch for c in x.clusters] )
    cgraph = torch.zeros(len(tree_batchG) + 1, max_cls_size).int()
    for v,attr in tree_batchG.nodes(data=True):
        bid = attr['batch_id']
        offset = graph_scope[bid][0]
        tree_batchG.nodes[v]['inter_label'] = inter_label = [(x + offset, y) for x,y in attr['inter_label']]
        tree_batchG.nodes[v]['cluster'] = cls = [x + offset for x in attr['cluster']]
        tree_batchG.nodes[v]['assm_cands'] = [add(x, offset) for x in attr['assm_cands']]
        cgraph[v, :len(cls)] = torch.IntTensor(cls)

    all_orders = []
    for i,hmol in enumerate(mol_batch):
        offset = tree_scope[i][0]
        order = [(x + offset, y + offset, z) for x,y,z in hmol.order[:-1]] + [(hmol.order[-1][0] + offset, None, 0)]
        all_orders.append(order)

    tree_tensors = tree_tensors[:4] + (cgraph, tree_scope)
    return (tree_batchG, graph_batchG), (tree_tensors, graph_tensors), all_orders

In [43]:
mol_batch = batches[0]
vocab = vocab

mol_batch = [MolGraph(x) for x in mol_batch]
tree_tensors,tree_batchG = MolGraph.tensorize_graph(mol_batch[0].mol_tree,vocab)
# tree_tensors, tree_batchG = MolGraph.tensorize_graph([x.mol_tree for x in mol_batch, vocab)
# tree_tensors

TypeError: object of type 'int' has no len()

In [35]:
max_cls_size = max([len(c) for x in mol_batch for c in x.clusters])

In [37]:
cgraph = torch.zeros(len(tree_batchG) + 1, max_cls_size).int()
cgraph

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)